# Identifies objects in picture files

In [5]:
import cv2
import numpy as np

darknet_directory = "../../resources/darknet/"
resources = "../../resources/"


# Load YOLOv3 model
weights = darknet_directory + "yolov3.weights"
cfg = darknet_directory + "yolov3.cfg"
coco_names = darknet_directory + "coco.names"

net = cv2.dnn.readNet(weights, cfg)
with open(coco_names, "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load image
img_src = resources + "testpics/" + "no_metadata_testpic_dogs.jpeg"
image = cv2.imread(img_src)
height, width = image.shape[:2]

# Prepare image for YOLO
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
output_layers = net.getUnconnectedOutLayersNames()
layer_outputs = net.forward(output_layers)

# Initialize lists for detected objects and bounding boxes
boxes = []
confidences = []
class_ids = []

# Process detections
for output in layer_outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:  # Confidence threshold
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Calculate top-left corner
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-max suppression to remove redundant boxes
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Format bounding boxes as requested
bounding_boxes = []
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        start_corner = [x, y]
        stop_corner = [x + w, y + h]
        bounding_boxes.append({
            "start_corner": start_corner,
            "stop_corner": stop_corner,
            "class": classes[class_ids[i]],
            "confidence": confidences[i]
        })

# Print bounding boxes
for box in bounding_boxes:
    print(box)


{'start_corner': [89, 30], 'stop_corner': [337, 280], 'class': 'dog', 'confidence': 0.9139481782913208}
{'start_corner': [367, 46], 'stop_corner': [655, 290], 'class': 'dog', 'confidence': 0.9941235184669495}
{'start_corner': [103, 316], 'stop_corner': [319, 498], 'class': 'dog', 'confidence': 0.9692137241363525}
{'start_corner': [388, 305], 'stop_corner': [657, 544], 'class': 'dog', 'confidence': 0.7733085751533508}
